## Gold Prices and Geopolitical Risk Analysis
### Project Overview

This project investigates the relationship between geopolitical risk and gold prices over time. Gold is traditionally considered a safe-haven asset, meaning investors may move into gold during periods of heightened political uncertainty or global instability.

Using daily gold spot prices and a Geopolitical Risk Index (GPR), this analysis aims to:

Examine whether increases in geopolitical risk are associated with changes in gold returns

Measure the statistical strength of the relationship

Explore lag effects to determine whether gold reacts immediately or with delay

Analyze volatility patterns during high-risk periods

The ultimate objective is to determine whether gold behaves as a true hedge against geopolitical uncertainty

In [11]:
import pandas as pd
import numpy as np 
import scipy
from matplotlib import pyplot as plt
import seaborn as sns 


In [12]:
gold_df = pd.read_csv('gold_spot_daily.csv')
gold_df.head()

,DATE,GOLD_PRICE,GOLD_OPEN,GOLD_HIGH,GOLD_LOW,GOLD_CHANGE_%
0,10-09-2025,3630.90,3633.61,3634.42,3620.90,-0.07
1,09-09-2025,3633.61,3637.10,3674.75,3625.33,-0.06
2,08-09-2025,3635.84,3586.82,3646.60,3579.67,1.24
3,07-09-2025,3591.19,3592.07,3596.56,3586.95,0.12
4,05-09-2025,3586.81,3547.00,3600.33,3540.05,1.15


In [13]:
# checking for basic data information 
def check_data(df):
  info = df.info()
  missing = df.isnull().sum()
  stastics = df.describe()
  print(info, missing, stastics)
check_data(gold_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10571 entries, 0 to 10570
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DATE           10571 non-null  object 
 1   GOLD_PRICE     10571 non-null  float64
 2   GOLD_OPEN      10571 non-null  float64
 3   GOLD_HIGH      10571 non-null  float64
 4   GOLD_LOW       10571 non-null  float64
 5   GOLD_CHANGE_%  10571 non-null  float64
dtypes: float64(5), object(1)
memory usage: 495.6+ KB
None DATE             0
GOLD_PRICE       0
GOLD_OPEN        0
GOLD_HIGH        0
GOLD_LOW         0
GOLD_CHANGE_%    0
dtype: int64          GOLD_PRICE     GOLD_OPEN     GOLD_HIGH      GOLD_LOW  GOLD_CHANGE_%
count  10571.000000  10571.000000  10571.000000  10571.000000   10571.000000
mean     887.276856    887.412518    892.378187    881.901980       0.028118
std      666.189751    665.535068    671.811142    659.998671       0.976355
min      252.550000    252.350000    252.55000

We can see that from the informatiom that the data has 6 columns and 10571 rows. There are no missing values in the gold dataset  as also observed

In [14]:
gpr_df = pd.read_csv('geopolitical_risk_index_daily.csv')
gpr_df.tail()

,DATE,GPRD,GPRD_ACT,GPRD_THREAT,EVENT
14856,04-09-2025,126.80,79.00,171.33,NaN
14857,05-09-2025,166.42,110.61,224.05,NaN
14858,06-09-2025,143.40,128.66,155.02,NaN
14859,07-09-2025,83.51,111.00,92.59,NaN
14860,08-09-2025,117.26,97.42,146.26,NaN


In [15]:
check_data(gpr_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14861 entries, 0 to 14860
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATE         14861 non-null  object 
 1   GPRD         14861 non-null  object 
 2   GPRD_ACT     14861 non-null  object 
 3   GPRD_THREAT  14861 non-null  float64
 4   EVENT        11 non-null     object 
dtypes: float64(1), object(4)
memory usage: 580.6+ KB
None DATE               0
GPRD               0
GPRD_ACT           0
GPRD_THREAT        0
EVENT          14850
dtype: int64         GPRD_THREAT
count  14861.000000
mean     105.015151
std       62.834017
min        0.000000
25%       64.530000
50%       92.160000
75%      130.260000
max      809.490000


From the information generated from the we can see that the datset has 5 columns and 14861 rows and that the `EVENT` column had 14850 missing values.


## Data Cleaning 


In [16]:
gpr_df.drop(columns='EVENT', axis=1, inplace=True)

In [17]:
gold_df["DATE"] = pd.to_datetime(gold_df["DATE"], dayfirst=True)
gpr_df["DATE"] = pd.to_datetime(gpr_df["DATE"], dayfirst=True)

In [18]:
# replacing the index with the dates
gold_df.set_index('DATE', inplace = True)
gpr_df.set_index('DATE', inplace = True)

In [19]:
# sorting dates to be in chronological order
gold_df.sort_index(inplace = True)
gpr_df.sort_index(inplace = True)

In [20]:
gold_df.head()

,GOLD_PRICE,GOLD_OPEN,GOLD_HIGH,GOLD_LOW,GOLD_CHANGE_%
DATE,,,,,
1985-01-02,305.5,305.5,305.5,305.5,-0.91
1985-01-03,302.3,302.3,302.3,302.3,-1.05
1985-01-04,303.1,303.1,303.1,303.1,0.26
1985-01-07,298.2,298.2,298.2,298.2,-1.62
1985-01-08,302.0,302.0,302.0,302.0,1.27


In [21]:
# merging the two datasets 
merged_df = gold_df.merge(gpr_df,
                          left_index=True,
                          right_index=True,
                          how = 'inner')
merged_df.head()

,GOLD_PRICE,GOLD_OPEN,GOLD_HIGH,GOLD_LOW,GOLD_CHANGE_%,GPRD,GPRD_ACT,GPRD_THREAT
DATE,,,,,,,,
1985-01-02,305.5,305.5,305.5,305.5,-0.91,115.68,146.77,87.44
1985-01-03,302.3,302.3,302.3,302.3,-1.05,97.43,158.94,29.46
1985-01-04,303.1,303.1,303.1,303.1,0.26,157.37,156.88,157.03
1985-01-07,298.2,298.2,298.2,298.2,-1.62,105.31,57.27,143.30
1985-01-08,302.0,302.0,302.0,302.0,1.27,87.40,34.85,130.82


In [22]:
check_data(merged_df)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10569 entries, 1985-01-02 to 2025-09-08
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   GOLD_PRICE     10569 non-null  float64
 1   GOLD_OPEN      10569 non-null  float64
 2   GOLD_HIGH      10569 non-null  float64
 3   GOLD_LOW       10569 non-null  float64
 4   GOLD_CHANGE_%  10569 non-null  float64
 5   GPRD           10569 non-null  object 
 6   GPRD_ACT       10569 non-null  object 
 7   GPRD_THREAT    10569 non-null  float64
dtypes: float64(6), object(2)
memory usage: 743.1+ KB
None GOLD_PRICE       0
GOLD_OPEN        0
GOLD_HIGH        0
GOLD_LOW         0
GOLD_CHANGE_%    0
GPRD             0
GPRD_ACT         0
GPRD_THREAT      0
dtype: int64          GOLD_PRICE     GOLD_OPEN     GOLD_HIGH      GOLD_LOW  GOLD_CHANGE_%  \
count  10569.000000  10569.000000  10569.000000  10569.000000   10569.000000   
mean     886.757417    886.892518    891.855488    

In [23]:
# check for missing duplicates 
merged_df.duplicated().sum()


0

In [24]:
merged_df.columns

Index(['GOLD_PRICE', 'GOLD_OPEN', 'GOLD_HIGH', 'GOLD_LOW', 'GOLD_CHANGE_%',
       'GPRD', 'GPRD_ACT', 'GPRD_THREAT'],
      dtype='object')

In [ ]:
# Renaming the columns for easy coding 
merged_df.rename(columns={
    "GOLD_PRICE": "gold_close",
    "GOLD_OPEN": "gold_open",
    "GOLD_HIGH": "gold_high",
    "GOLD_LOW": "gold_low",
    "GOLD_CHANGE_%": "gold_change_pct",
    "GPRD": "gpr_index",
    "GPRD_ACT": "gpr_action",
    "GPRD_THREAT": "gpr_threat"
}, inplace=True)

In [26]:
merged_df.columns

Index(['gold_close', 'gold_open', 'gold_high', 'gold_low', 'gold_change_pct',
       'gpr_index', 'gpr_action', 'gpr_threat'],
      dtype='object')

In [30]:
# calcuting the log gold returns for time series analysis
merged_df['log_price_change'] = np.log(merged_df['gold_close'] / merged_df['gold_close'].shift(1))

In [36]:
merged_df.tail()

,gold_close,gold_open,gold_high,gold_low,gold_change_pct,gpr_index,gpr_action,gpr_threat,log_price_change
DATE,,,,,,,,,
2025-09-03,3559.43,3535.69,3578.80,3526.26,0.73,209.95,232.56,193.98,0.007246
2025-09-04,3546.17,3560.57,3564.45,3511.19,-0.37,126.80,79.00,171.33,-0.003732
2025-09-05,3586.81,3547.00,3600.33,3540.05,1.15,166.42,110.61,224.05,0.011395
2025-09-07,3591.19,3592.07,3596.56,3586.95,0.12,83.51,111.00,92.59,0.001220
2025-09-08,3635.84,3586.82,3646.60,3579.67,1.24,117.26,97.42,146.26,0.012357


In [32]:
#drop the first NaN caused by the shift
merged_df.dropna(inplace=True)

## Explatory Data Anylysis(EDA)